In [ ]:
# Cài đặt các thư viện cần thiết (bỏ qua nếu môi trường đã có)
#!pip install -U scikit-learn
%pip install tqdm
%pip install pandas
%pip install seaborn
%pip install numpy


In [ ]:
# Tắt cảnh báo để output gọn gàng (cân nhắc bật lại khi cần debug)
# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn



In [ ]:
# Thư viện hiển thị tiến trình
from tqdm import tqdm

# Số học, dữ liệu, vẽ
import numpy as np
import pandas as pd
from itertools import accumulate  # không dùng, có thể bỏ nếu muốn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Scikit-learn: tiền xử lý, mô hình, đánh giá, pipeline, tìm kiếm tham số
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Cấu hình hiển thị
sns.set_context('notebook')
sns.set_style('white')

# Tắt cảnh báo không quan trọng
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [ ]:
# Hàm trợ giúp: tính và vẽ R^2 theo từng biến đơn lẻ cho một mô hình (fit 1-D)
# Lưu ý: đoạn remove("three") giả định có cột tên "three"; nếu không có sẽ lỗi
# Có thể bỏ dòng đó hoặc thêm kiểm tra an toàn

def get_R2_features(model,test=True): 
    features=list(X)
    if "three" in features:
        features.remove("three")
    
    R_2_train=[]
    R_2_test=[]

    for feature in features:
        model.fit(X_train[[feature]],y_train)
        R_2_test.append(model.score(X_test[[feature]],y_test))
        R_2_train.append(model.score(X_train[[feature]],y_train))
        
    plt.bar(features,R_2_train,label="Train")
    plt.bar(features,R_2_test,label="Test")
    plt.xticks(rotation=90)
    plt.ylabel("$R^2$")
    plt.legend()
    plt.show()
    print("Training R^2 mean value {} Testing R^2 mean value {} ".format(str(np.mean(R_2_train)),str(np.mean(R_2_test))) )
    print("Training R^2 max value {} Testing R^2 max value {} ".format(str(np.max(R_2_train)),str(np.max(R_2_test))) )

In [ ]:
def  plot_dis(y,yhat):
    
    plt.figure()
    ax1 = sns.distplot(y, hist=False, color="r", label="Actual Value")
    sns.distplot(yhat, hist=False, color="b", label="Fitted Values" , ax=ax1)
    plt.legend()

    plt.title('Actual vs Fitted Values')
    plt.xlabel('Price (in dollars)')
    plt.ylabel('Proportion of Cars')

    plt.show()
    plt.close()

In [ ]:
# Hàm tải file trong môi trường Pyodide (bất đồng bộ)
from pyodide.http import pyfetch

async def download(url, filename):
    response = await pyfetch(url)
    if response.status == 200:
        with open(filename, "wb") as f:
            f.write(await response.bytes())

In [ ]:
# Đường dẫn dữ liệu được mã hóa sẵn (đã one-hot)
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML240EN-SkillsNetwork/labs/encoded_car_data.csv'

# Nếu chạy local đã có file, có thể bỏ qua dòng dưới
await download(URL, 'encoded_car_data.csv')

# Đọc vào DataFrame
data = pd.read_csv('encoded_car_data.csv')

In [ ]:
# Kiểm tra thông tin cột và kiểu dữ liệu
data.info()

In [ ]:
# Hồi quy bậc 2 giữa curbweight và price để quan sát xu hướng phi tuyến
sns.lmplot(x = 'curbweight', y = 'price', data = data, order=2)

In [ ]:
# Hồi quy bậc 2 giữa carlength và price
sns.lmplot(x = 'carlength', y = 'price', data = data, order=2)

In [ ]:
# Hồi quy bậc 2 giữa horsepower và price
sns.lmplot(x = 'horsepower', y = 'price', data = data, order=2)

In [ ]:
# Tách đặc trưng X và mục tiêu y
X = data.drop('price', axis=1)
y = data.price

In [ ]:
# Chia train/test với tỉ lệ 70/30, cố định seed để tái lập
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)
print("Number of test samples:", X_test.shape[0])
print("Number of training samples:", X_train.shape[0])


In [ ]:
# Mô hình hồi quy tuyến tính baseline
lm = LinearRegression()

In [ ]:
# Huấn luyện trên tập train
lm.fit(X_train, y_train)

In [ ]:
# Dự đoán trên tập test
predicted = lm.predict(X_test)

In [ ]:
# Đánh giá R^2 trên train và test để so sánh mức độ khớp
print("R^2 on training  data ",lm.score(X_train, y_train))
print("R^2 on testing data ",lm.score(X_test,y_test))

In [ ]:
# Phân phối thực tế vs dự đoán để kiểm tra sai lệch
plot_dis(y_test,predicted)

In [ ]:
# Từ điển hệ số ước lượng theo từng biến
{col:coef for col,coef in zip(X.columns, lm.coef_)}

In [ ]:
# Biểu đồ độ lớn tuyệt đối của hệ số (bỏ 2 cột đầu nếu không cần)
plt.bar(X.columns[2:],abs(lm.coef_[2:]))
plt.xticks(rotation=90)
plt.ylabel("$coefficients$")
plt.show()

In [ ]:
# Vẽ R^2 theo từng biến đơn lẻ trên train/test để so sánh sức mạnh dự báo
get_R2_features(lm)

In [ ]:
# Pipeline có chuẩn hóa -> hồi quy tuyến tính
pipe = Pipeline([('ss',StandardScaler() ),('lr', LinearRegression())])
pipe.fit(X_train,y_train)
print("R^2 on training  data ", pipe.score(X_train, y_train))
print("R^2 on testing data ", pipe.score(X_test,y_test))
predicted = pipe.predict(X_test)
plot_dis(y_test,predicted)

In [ ]:
# So sánh R^2 từng biến khi dùng pipeline chuẩn hóa + LR
pipe = Pipeline([('ss',StandardScaler() ),('lr', LinearRegression())])
get_R2_features(pipe)

In [ ]:
# Tạo đặc trưng đa thức bậc 2, bỏ hằng số (include_bias=False)
poly_features = PolynomialFeatures(degree=2, include_bias=False)

In [ ]:
# Biến đổi train/test sang không gian đa thức
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)

In [ ]:
# Số chiều sau khi mở rộng đa thức
print(X_train_poly.shape)


In [ ]:
# Số chiều của tập test sau khi mở rộng
print(X_test_poly.shape)

In [ ]:
# Huấn luyện LinearRegression trên đặc trưng đa thức
lm = LinearRegression()
lm.fit(X_train_poly, y_train)

In [ ]:
# Dự đoán trên train để đánh giá mức độ khớp
predicted = lm.predict(X_train_poly)

In [ ]:
# So sánh R^2 train/test cho mô hình đa thức để phát hiện overfitting
print("R^2 on training data:", lm.score(X_train_poly, y_train))
print("R^2 on testing data:", lm.score(X_test_poly,y_test))

In [ ]:
# Định nghĩa pipeline: PolynomialFeatures -> LinearRegression
Input=[ ('polynomial', PolynomialFeatures(include_bias=False,degree=2)), ('model', LinearRegression())]

In [ ]:
# Khởi tạo và huấn luyện pipeline đa thức
pipe=Pipeline(Input)
pipe.fit(X_train, y_train)

In [ ]:
# Đánh giá pipeline đa thức
print("R^2 on training  data:", pipe.score(X_train, y_train))
print("R^2 on testing data:", pipe.score(X_test,y_test))

In [ ]:
# R^2 từng biến khi dùng pipeline đa thức
get_R2_features(pipe)

In [ ]:
# Pipeline đa thức + chuẩn hóa + hồi quy
Input=[ ('polynomial', PolynomialFeatures(include_bias=False,degree=2)), ('scaler', StandardScaler()),('model',LinearRegression() )]
pipe=Pipeline(Input)

In [ ]:
# Lưới tham số: thử bậc đa thức 1-3
param_grid = {
    "polynomial__degree": [1, 2, 3],
}

In [ ]:
# Cấu hình GridSearchCV (có thể tăng n_jobs để tận dụng đa lõi)
search = GridSearchCV(pipe, param_grid, n_jobs=1)


In [ ]:
# Fit pipeline trên train
pipe.fit(X_train, y_train)

In [ ]:
# Lưu ý: thường sẽ fit trên train và đánh giá trên validation; ở đây demo dùng test
search.fit(X_test, y_test)

In [ ]:
# Mô hình tốt nhất theo GridSearchCV
best=search.best_estimator_
best

In [ ]:
# Điểm R^2 của mô hình tốt nhất
best.score(X_test,y_test)

In [ ]:
# Phân phối thực tế vs dự đoán của mô hình tốt nhất
predicted=best.predict(X_test)
plot_dis(y_test,predicted)

In [ ]:
# Tìm bậc đa thức tối ưu cho TỪNG biến đơn lẻ bằng GridSearchCV
# Lưu ý: ví dụ này fit trên tập test cho nhanh; thực hành chuẩn nên fit trên train/validation
features=list(X)

R_2_train=[]
R_2_test=[]

for feature in features:
    # Thử nhiều bậc đa thức và tùy chọn ràng buộc hệ số dương
    param_grid = {
        "polynomial__degree": [1, 2, 3, 4, 5],
        "model__positive": [True, False]
    }
    Input=[('polynomial', PolynomialFeatures(include_bias=False, degree=2)),
           ('model', LinearRegression())]
    pipe=Pipeline(Input)
    print(feature)
    search = GridSearchCV(pipe, param_grid, n_jobs=2)
    search.fit(X_test[[feature]], y_test)
    best=search.best_estimator_

    # Ghi nhận R^2 trên test và train cho biến đơn lẻ
    R_2_test.append(best.score(X_test[[feature]], y_test))
    R_2_train.append(best.score(X_train[[feature]], y_train))

# Vẽ so sánh R^2 Train/Test theo từng biến
plt.bar(features, R_2_train, label="Train")
plt.bar(features, R_2_test, label="Test")
plt.xticks(rotation=90)
plt.ylabel("$R^2$")
plt.legend()
plt.show()
print("Training R^2 mean value {} Testing R^2 mean value {} ".format(str(np.mean(R_2_train)), str(np.mean(R_2_test))))
print("Training R^2 max value {} Testing R^2 max value {} ".format(str(np.max(R_2_train)), str(np.max(R_2_test))))